In [ ]:
pip install pycryptodome

In [ ]:
pip install PyQt5 pycryptodome


In [ ]:
import sys
import time
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel, QLineEdit

# --- Playfair Cipher ---
def generate_playfair_matrix(key):
    key = "".join(dict.fromkeys(key.upper().replace("J", "I")))
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
    key += "".join(filter(lambda c: c not in key, alphabet))
    return [list(key[i:i+5]) for i in range(0, 25, 5)]

def find_position(matrix, char):
    for i, row in enumerate(matrix):
        if char in row:
            return i, row.index(char)
    return None

def playfair_encrypt(text, key):
    matrix = generate_playfair_matrix(key)
    text = text.upper().replace("J", "I").replace(" ", "")
    if len(text) % 2 != 0:
        text += "X"  # Padding if odd length

    ciphertext = ""
    for i in range(0, len(text), 2):
        a, b = text[i], text[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2:
            ciphertext += matrix[r1][(c1 + 1) % 5] + matrix[r2][(c2 + 1) % 5]
        elif c1 == c2:
            ciphertext += matrix[(r1 + 1) % 5][c1] + matrix[(r2 + 1) % 5][c2]
        else:
            ciphertext += matrix[r1][c2] + matrix[r2][c1]

    return ciphertext

def playfair_decrypt(ciphertext, key):
    matrix = generate_playfair_matrix(key)
    plaintext = ""

    for i in range(0, len(ciphertext), 2):
        a, b = ciphertext[i], ciphertext[i+1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2:
            plaintext += matrix[r1][(c1 - 1) % 5] + matrix[r2][(c2 - 1) % 5]
        elif c1 == c2:
            plaintext += matrix[(r1 - 1) % 5][c1] + matrix[(r2 - 1) % 5][c2]
        else:
            plaintext += matrix[r1][c2] + matrix[r2][c1]

    return plaintext

# --- Rail Fence Cipher ---
def rail_fence_encrypt(text, depth):
    if depth <= 1 or depth >= len(text):
        return text  # No encryption if depth is invalid

    rail = [["\n" for _ in range(len(text))] for _ in range(depth)]
    dir_down = False
    row, col = 0, 0

    for char in text:
        if row == 0 or row == depth - 1:
            dir_down = not dir_down
        rail[row][col] = char
        col += 1
        row += 1 if dir_down else -1

    return "".join(c for row in rail for c in row if c != "\n")

def rail_fence_decrypt(ciphertext, depth):
    if depth <= 1 or depth >= len(ciphertext):  
        return ciphertext  # No encryption applied in these cases

    rail = [['\n' for _ in range(len(ciphertext))] for _ in range(depth)]
    dir_down = None
    row, col = 0, 0

    # First, mark positions in the rail matrix
    for i in range(len(ciphertext)):
        if row == 0:
            dir_down = True
        elif row == depth - 1:
            dir_down = False

        rail[row][col] = '*'
        col += 1
        row += 1 if dir_down else -1

    # Fill marked positions with ciphertext characters
    index = 0
    for i in range(depth):
        for j in range(len(ciphertext)):
            if rail[i][j] == '*' and index < len(ciphertext):
                rail[i][j] = ciphertext[index]
                index += 1

    # Read the decrypted text by following the zigzag pattern
    result = []
    row, col = 0, 0
    for i in range(len(ciphertext)):
        if row == 0:
            dir_down = True
        elif row == depth - 1:
            dir_down = False

        if rail[row][col] != '*':  # Ensure it's a valid character
            result.append(rail[row][col])
        col += 1
        row += 1 if dir_down else -1

    return ''.join(result)

# --- GUI Implementation for Analysis ---
class CipherAnalysisApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
    
    def initUI(self):
        layout = QVBoxLayout()
        self.input_text = QLineEdit(self)
        layout.addWidget(QLabel("Enter Plaintext:"))
        layout.addWidget(self.input_text)

        self.key_input = QLineEdit(self)
        layout.addWidget(QLabel("Enter Key:"))
        layout.addWidget(self.key_input)

        self.result_text = QTextEdit(self)
        layout.addWidget(QLabel("Result:"))
        layout.addWidget(self.result_text)

        encrypt_button = QPushButton("Analyze Encryption")
        encrypt_button.clicked.connect(self.analyze_encryption)
        layout.addWidget(encrypt_button)

        self.setLayout(layout)
        self.setWindowTitle("Cipher Analysis GUI")
        self.show()
    
    def analyze_encryption(self):
        plaintext = self.input_text.text().strip()
        key = self.key_input.text().strip()
        depth = 3  # Rail Fence depth

        if not plaintext or not key:
            self.result_text.setText("Error: Please enter both plaintext and key.")
            return
        
        # Encryption Process
        start_time = time.time()
        playfair_enc = playfair_encrypt(plaintext, key)
        rail_enc = rail_fence_encrypt(playfair_enc, depth)
        encryption_time = time.time() - start_time

        # Decryption Process
        start_time = time.time()
        rail_dec = rail_fence_decrypt(rail_enc, depth)
        playfair_dec = playfair_decrypt(rail_dec, key)
        decryption_time = time.time() - start_time

        self.result_text.setText(f"Playfair Encrypted: {playfair_enc}\nRail Fence Encrypted: {rail_enc}\n"
                                 f"Decryption: {playfair_dec}\n"
                                 f"Encryption Time: {encryption_time:.6f} sec\n"
                                 f"Decryption Time: {decryption_time:.6f} sec")

# --- Main Execution ---
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = CipherAnalysisApp()
    sys.exit(app.exec_())


In [ ]:
import sys
import time
import base64
import random
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel, QLineEdit
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, DES3
from Crypto.Random import get_random_bytes

# --- RSA Key Exchange ---
def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

def rsa_encrypt(public_key, message):
    recipient_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(recipient_key)
    return cipher_rsa.encrypt(message)

def rsa_decrypt(private_key, ciphertext):
    private_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(private_key)
    return cipher_rsa.decrypt(ciphertext)

# --- Triple DES Encryption ---
def get_valid_des3_key(key):
    while len(key) < 24:
        key += key  
    return key[:24].encode()

def triple_des_encrypt(key, plaintext):
    cipher = DES3.new(key, DES3.MODE_ECB)
    while len(plaintext) % 8 != 0:
        plaintext += " "
    return cipher.encrypt(plaintext.encode())

def triple_des_decrypt(key, ciphertext):
    cipher = DES3.new(key, DES3.MODE_ECB)
    return cipher.decrypt(ciphertext).decode().strip()

# --- Simulating Bit Errors ---
def introduce_bit_error(ciphertext):
    corrupted = bytearray(ciphertext)
    index = random.randint(0, len(ciphertext) - 1)
    corrupted[index] ^= 1  # Flip one bit
    return bytes(corrupted)

# --- GUI Implementation ---
class HybridCipherApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
    
    def initUI(self):
        layout = QVBoxLayout()
        self.input_text = QLineEdit(self)
        layout.addWidget(QLabel("Enter Plaintext:"))
        layout.addWidget(self.input_text)
        self.key_input = QLineEdit(self)
        layout.addWidget(QLabel("Enter Key:"))
        layout.addWidget(self.key_input)
        self.result_text = QTextEdit(self)
        layout.addWidget(QLabel("Result:"))
        layout.addWidget(self.result_text)
        encrypt_button = QPushButton("Encrypt & Simulate Transmission", self)
        encrypt_button.clicked.connect(self.encrypt_text)
        layout.addWidget(encrypt_button)
        self.setLayout(layout)
        self.setWindowTitle("Hybrid Cipher GUI")
        self.show()
    
    def encrypt_text(self):
        plaintext = self.input_text.text().strip()
        key = self.key_input.text().strip()
        if not plaintext or not key:
            self.result_text.setText("Error: Please enter both plaintext and key.")
            return
        
        # Step 1: RSA Key Exchange
        rsa_private, rsa_public = generate_rsa_keys()
        secret_key = get_random_bytes(16)
        encrypted_secret = rsa_encrypt(rsa_public, secret_key)
        decrypted_secret = rsa_decrypt(rsa_private, encrypted_secret)
        
        # Step 2: Triple DES Encryption
        des3_key = get_valid_des3_key(key)
        triple_des_encrypted = triple_des_encrypt(des3_key, plaintext)
        
        # Step 3: Simulate bit error during transmission
        corrupted_ciphertext = introduce_bit_error(triple_des_encrypted)
        
        # Step 4: Attempt decryption with bit error
        try:
            triple_des_decrypted = triple_des_decrypt(des3_key, corrupted_ciphertext)
        except Exception as e:
            triple_des_decrypted = f"Decryption failed due to bit error: {str(e)}"
        
        # Display results
        self.result_text.setText(
            f"Triple DES Encrypted (Base64): {base64.b64encode(triple_des_encrypted).decode()}\n"
            f"Triple DES Decrypted (Normal): {triple_des_decrypt(des3_key, triple_des_encrypted)}\n"
            f"Triple DES Decrypted (After Bit Error): {triple_des_decrypted}"
        )

# --- Main Execution ---
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = HybridCipherApp()
    sys.exit(app.exec_())
